# Projeto: Detecção de Spam em Mensagens de Texto

🎯 Objetivo:

Construir um modelo que identifica se uma mensagem é "spam" ou "ham" (normal), usando NLP + ML.

#### Imports 

In [3]:
import pandas as pd
import string
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS, TfidfVectorizer
from sklearn.model_selection import train_test_split, cross_val_score 
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, accuracy_score

#### Base de Dados

In [5]:
# Local da base de dados
url = 'https://raw.githubusercontent.com/justmarkham/DAT8/master/data/sms.tsv'

# criando data freme
df = pd.read_csv(url, sep='\t', header=None, names=['label', 'message'])

df.head(3)

,label,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...


In [6]:
# porcentagem de spam vs ham
print(df['label'].value_counts(normalize=True)) 

label
ham     0.865937
spam    0.134063
Name: proportion, dtype: float64


## pré-processando e Modelo

In [8]:
# criando função que remove STOP WORDS (palavras de parada), pontuações e etc
def limpar_texto_sem_nltk(texto):
    texto = texto.lower()
    texto = texto.translate(str.maketrans('', '', string.punctuation))
    palavras = texto.split()
    palavras_filtradas = [palavra for palavra in palavras if palavra not in ENGLISH_STOP_WORDS]
    return ' '.join(palavras_filtradas)

# criando nova coluna com os textos limpos no data frame
df['texto_limpo'] = df['message'].apply(limpar_texto_sem_nltk)

print(df[['message', 'texto_limpo']].head())

                                             message  \
0  Go until jurong point, crazy.. Available only ...   
1                      Ok lar... Joking wif u oni...   
2  Free entry in 2 a wkly comp to win FA Cup fina...   
3  U dun say so early hor... U c already then say...   
4  Nah I don't think he goes to usf, he lives aro...   

                                         texto_limpo  
0  jurong point crazy available bugis n great wor...  
1                            ok lar joking wif u oni  
2  free entry 2 wkly comp win fa cup final tkts 2...  
3                        u dun say early hor u c say  
4                      nah dont think goes usf lives  


In [9]:
# fazendo a vetorização pro modelo
vectorizer = TfidfVectorizer()  
X = vectorizer.fit_transform(df['texto_limpo'])

In [10]:
# variável alvo label
y = df['label']  

# separando treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [11]:
# treinando modelo 
model = MultinomialNB()
model.fit(X_train, y_train)

MultinomialNB()

## Avaliando modelo

In [13]:
# Fazendo a previsão dos rótulos (spam ou ham)
y_pred = model.predict(X_test)

# Imprime o relatório detalhado da classificação
print(classification_report(y_test, y_pred))

# Imprime a acurácia geral do modelo no conjunto de teste
print("Acurácia:", accuracy_score(y_test, y_pred))

              precision    recall  f1-score   support

         ham       0.96      1.00      0.98       966
        spam       1.00      0.74      0.85       149

    accuracy                           0.97      1115
   macro avg       0.98      0.87      0.92      1115
weighted avg       0.97      0.97      0.96      1115

Acurácia: 0.9659192825112107


In [14]:
# Calcula a acurácia do modelo no conjunto de treinamento
train_acc = model.score(X_train, y_train)

# Calcula a acurácia do modelo no conjunto de teste
test_acc = model.score(X_test, y_test)

# Imprime as acurácias de treinamento e teste com 4 casas decimais
print(f'Train acc: {train_acc:.4f}, Test acc: {test_acc:.4f}')

Train acc: 0.9746, Test acc: 0.9659


In [15]:
# Realiza validação cruzada com 5 folds para avaliar o desempenho do modelo
scores = cross_val_score(model, X, y, cv=5)

# Imprime a média e o desvio padrão das acurácias obtidas na validação cruzada
print(f'CV accuracy: {scores.mean():.4f} ± {scores.std():.4f}')

CV accuracy: 0.9623 ± 0.0042


Conclusão:

O modelo de detecção de spam apresentou um ótimo desempenho, alcançando alta acurácia tanto nos dados de treino quanto nos de teste, 
além de manter resultados consistentes na validação cruzada. 

Isso indica que o modelo conseguiu aprender bem os padrões das mensagens sem depender demais de exemplos específicos.

Ou seja, ele é capaz de identificar novas mensagens como spam ou não spam de forma confiável e equilibrada.